In [7]:
from scai.custom_chat_models.crfm import crfmChatLLM
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory

# meta prompt 
from scai.modules.meta_prompt.base import MetaPromptModel

In [8]:
buffer = CustomConversationBufferWindowMemory(k=1)
from langchain.schema import SystemMessage
buffer.save_context(system={"system": "You are a helpful assistant that always tells the truth."}, 
                    user={"user": "Write a 50 word essay about wildlife in africa."}, 
                    assistant={"assistant": "Africa's iconic wildlife, notably lions, elephants, giraffes, and rhinoceroses, embodies diverse biodiversity. \
                               These majestic beings, with giraffes towering above, underpin ecological harmony and bolster tourism. \
                               Yet, they face peril from poaching and habitat destruction. It's essential to intensify conservation measures to safeguard Africa's irreplaceable wildlife legacy."})


buffer.save_context(user={"user": "Please revise the essay to exclude specific mentions of lions, elephants, and giraffes, but still convey the importance and diversity of large African wildlife."})

meta_model = MetaPromptModel(llm=chat_llm)
response = meta_model.run(name="meta_prompt_1", buffer=buffer)

In [2]:
# prompting a custom chat model
from scai.custom_chat_models.crfm import crfmChatLLM

CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
                   crfm_api_key=CRFM_API_KEY, 
                   max_tokens=100, # need to be careful with this one
                   num_completions=1,
                   request_timeout=10,
                   verbose=False,
                   temperature=0.9,
                   )


In [34]:
# buffer = CustomConversationBufferWindowMemory(k=1)
# assistant_system_message = None

# user_system_message = "You are really scared of giraffes, lions and elephants and do not want to learn about them"

# tasks = [{"Write a 100 word essay about wildlife in africa, especially large anmials."}]

# user_samples_task = tasks[0]

# user_samples_task

In [4]:
buffer = CustomConversationBufferWindowMemory(k=1)
from langchain.schema import SystemMessage
buffer.save_context(system={"system": "You are a helpful assistant that always tells the truth."}, 
                    user={"user": "Write a 50 word essay about wildlife in africa."}, 
                    assistant={"assistant": "Africa's iconic wildlife, notably lions, elephants, giraffes, and rhinoceroses, embodies diverse biodiversity. \
                               These majestic beings, with giraffes towering above, underpin ecological harmony and bolster tourism. \
                               Yet, they face peril from poaching and habitat destruction. It's essential to intensify conservation measures to safeguard Africa's irreplaceable wildlife legacy."})


buffer.save_context(user={"user": "Please revise the essay to exclude specific mentions of lions, elephants, and giraffes, but still convey the importance and diversity of large African wildlife."})

meta_model = MetaPromptModel(llm=chat_llm)
response = meta_model.run(name="meta_prompt_1", buffer=buffer)

In [5]:
print(response)

system: As a helpful assistant that always tells the truth, always adapt your responses to accommodate the user's specific requests. Be mindful of their preferences and provide information or revisions as needed while maintaining focus on the topic. In the case of discussing large African wildlife, ensure you address the diversity and importance without explicitly mentioning the species they want to exclude.
